# K-means example

**算法原理**

kmeans的计算方法如下：

1. 随机选取k个中心点

2. 遍历所有数据，将每个数据划分到最近的中心点中

3. 计算每个聚类的平均值，并作为新的中心点

4. 重复2-3，直到这k个中线点不再变化（收敛了），或执行了足够多的迭代

In [1]:
from __future__ import print_function

import tensorflow as tf
import numpy as np
from tensorflow.contrib.factorization import KMeans

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data/', one_hot=True)
X_train = mnist.train.images

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
# patameters
n_steps = 50
batch_size = 1024
k = 25
n_classes = 10
n_features = 784

In [5]:
X = tf.placeholder(tf.float32, shape=[None, n_features])
Y = tf.placeholder(tf.float32, shape=[None, n_classes])

kmeans = KMeans(inputs=X, num_clusters=k, distance_metric='cosine', use_mini_batch=True)

In [9]:
# build k-means graph
all_scores, cluster_idx, scores, cluster_centers_initialized, init_op, train_op = kmeans.training_graph()
cluster_idx = cluster_idx[0]
avg_distance = tf.reduce_mean(scores)

In [10]:
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init, feed_dict={X: X_train})
sess.run(init_op, feed_dict={X: X_train})

# training
for i in range(1, n_steps + 1):
    _, dist, idx = sess.run([train_op, avg_distance, cluster_idx], feed_dict={X: X_train})
    if(i % 10 == 0 or i == 1):
        print('step: %i \t avg distance: %f' % (i, dist))

step: 1 	 avg distance: 0.341471
step: 10 	 avg distance: 0.221609
step: 20 	 avg distance: 0.220328
step: 30 	 avg distance: 0.219776
step: 40 	 avg distance: 0.219419
step: 50 	 avg distance: 0.219154


In [14]:
counts = np.zeros(shape=(k, n_classes))
for i in range(len(idx)):
    counts[idx[i]] += mnist.train.labels[i]

labels_map = [np.argmax(c) for c in counts]
labels_map = tf.convert_to_tensor(labels_map)

cluster_labels = tf.nn.embedding_lookup(labels_map, cluster_idx)

correct_prediction = tf.equal(cluster_labels, tf.cast(tf.argmax(Y, 1), tf.int32))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

test_x, test_y = mnist.test.images, mnist.test.labels
print('accuracy: ', sess.run(accuracy_op, feed_dict={X: test_x, Y: test_y}))

accuracy:  0.7127
